# Fashion-MNIST with tf.keras and tf.data

In this notebook, you'll learn how create a `tf.data` [Dataset](https://www.tensorflow.org/api_docs/python/tf/data/Dataset) for the Fashion-MNIST dataset. The `tf.data` API provides tools for working with data (including common functionality, like shuffling, batching, etc), as well as high performance utilities (like parallel reads, and pre-fetching to GPUs) - should you need these down the road. Here, we'll wrap Fashion-MNIST with `tf.data`, just to introduce the API. Later, you'll learn how to use `tf.data` on your own data, say, to read a directory of images from disk or S3 bucket.

To learn more about `tf.data`, you can check out [this](https://www.youtube.com/watch?v=EHHdyM3NNiA&index=26&list=PLQY2H8rRoyvxjVx3zfw4vA4cvlKogyLNN) talk from the [2018 TensorFlow Developer Summit](https://www.tensorflow.org/dev-summit/).

The rest of the code similar to the previous notebook, with three changes:

1. Instead of using `model.fit`, we use `model.train_on_batch` as we iterate over our dataset. 
2. We will create a TensorFlow optimizer to pass to the Keras model.

### Eager execution
Eager execution is a mode for running TensorFlow that works just like regular Python. To learn more about eager, check out [this talk](https://www.youtube.com/watch?v=T8AW0fKP0Hs&list=PLQY2H8rRoyvxjVx3zfw4vA4cvlKogyLNN&index=8) from the same summit. In short Eager execution  is an imperative programming environment that evaluates operations immediately. TensorFlow 2 runs with eager execution by default, which is supported by `tf.keras`, and this is super useful for inspecting your program and debugging it. All the `tf.keras` model-building APIs are compatible with eager execution!

So to get started let's create our model from the last notebook:

In [1]:
import tensorflow as tf

import numpy as np

In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.fashion_mnist.load_data()

In [3]:
TRAINING_SIZE = len(train_images)
TEST_SIZE = len(test_images)

# Reshape from (N, 28, 28) to (N, 28*28=784)
train_images = np.reshape(train_images, (TRAINING_SIZE, 784))
test_images = np.reshape(test_images, (TEST_SIZE, 784))

# Convert the array to float32 as opposed to uint8
train_images = train_images.astype(np.float32)
test_images = test_images.astype(np.float32)

# Convert the pixel values from integers between 0 and 255 to floats between 0 and 1
train_images /= 255
test_images /=  255

In [4]:
NUM_CAT = 10

print("Before", train_labels[0]) # The format of the labels before conversion

train_labels  = tf.keras.utils.to_categorical(train_labels, NUM_CAT)

print("After", train_labels[0]) # The format of the labels after conversion

test_labels = tf.keras.utils.to_categorical(test_labels, NUM_CAT)

Before 9
After [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]


In [5]:
# Cast the labels to floats, needed later
train_labels = train_labels.astype(np.float32)
test_labels = test_labels.astype(np.float32)

In [6]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(512, activation=tf.nn.relu, input_shape=(784,)))
model.add(tf.keras.layers.Dense(NUM_CAT, activation=tf.nn.softmax))


# Create a Keras version optimizer
optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.001)

# We will now compile and print out a summary of our model
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


### Step 1) Create a tf.data Dataset

Here, we'll use the `tf.data.Dataset` [API](https://www.tensorflow.org/api_docs/python/tf/data) to convert the Numpy arrays into a TensorFlow dataset.

Next, we will create a simple `for` loop that will serve as our introduction into creating custom training loops. Although this essentially does the same thing as `model.fit` it allows us to get creative and customize the overall training process (should you like to, if you venture into research) and collect different metrics throughout the process.

In [7]:
BATCH_SIZE=128

# Because tf.data may work with potentially **large** collections of data
# we do not shuffle the entire dataset by default
# Instead, we maintain a buffer of SHUFFLE_SIZE elements
# and sample from there.
SHUFFLE_SIZE = 10000 

# Create the dataset
dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
dataset = dataset.shuffle(SHUFFLE_SIZE)
dataset = dataset.batch(BATCH_SIZE)

### Step 2) Iterate over the dataset
Here, we'll iterate over the dataset, and train our model using `model.train_on_batch`. To learn more about the elements returned from the dataset, you can print them out and try the `.numpy()` method.


In [8]:
EPOCHS=5

for epoch in range(EPOCHS):
    for images, labels in dataset:
        train_loss, train_accuracy = model.train_on_batch(images, labels)
  
    # Here you can gather any metrics or adjust your training parameters
    print('Epoch #%d\t Loss: %.6f\tAccuracy: %.6f' % (epoch + 1, train_loss, train_accuracy))

Epoch #1	 Loss: 0.496344	Accuracy: 0.802083
Epoch #2	 Loss: 0.427685	Accuracy: 0.822917
Epoch #3	 Loss: 0.380092	Accuracy: 0.833333
Epoch #4	 Loss: 0.365240	Accuracy: 0.843750
Epoch #5	 Loss: 0.300417	Accuracy: 0.864583


In [9]:
loss, accuracy = model.evaluate(test_images, test_labels)
print('Test accuracy: %.2f' % (accuracy))

10000/10000==============================] - 0s 48us/sample - loss: 0.3557 - acc: 0.8688
Test accuracy: 0.87


### Congratulations
You have trained a model on Fashion-MNIST using Keras and `tf.data`.